In [63]:
url = "https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files"
!wget $url/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget $url/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

(Reading database ... 155665 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [64]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import pickle

from statsmodels.formula.api import logit

# Techinical Indicators
import talib as ta

# Machine learning
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [65]:
# reading CSV file
df=pd.read_csv("HistoricalData_AMZN.csv")

In [66]:
# seting date as a index
#df=pd.read_csv(".\HistoricalData_AMZN.csv",header=0,index_col="Date",parse_dates=True)
df=pd.read_csv("HistoricalData_AMZN.csv")
# understanding the data
print(df.head())
# set the date as the index for hte data
df=df.set_index(pd.DatetimeIndex(df['Date'].values))
#Give the index a name
df.index.name="Date"
#Show the data
print(df.head())

       Date  Close/Last   Volume     Open     High      Low
0  6/9/2021     3281.15  2455530  3272.87  3297.58  3270.70
1  6/8/2021     3264.11  3416730  3222.61  3279.53  3218.01
2  6/7/2021     3198.01  2215756  3197.33  3208.00  3172.20
3  6/4/2021     3206.22  2249749  3212.00  3221.00  3198.81
4  6/3/2021     3187.01  2398293  3204.23  3214.44  3184.03
                Date  Close/Last   Volume     Open     High      Low
Date                                                                
2021-06-09  6/9/2021     3281.15  2455530  3272.87  3297.58  3270.70
2021-06-08  6/8/2021     3264.11  3416730  3222.61  3279.53  3218.01
2021-06-07  6/7/2021     3198.01  2215756  3197.33  3208.00  3172.20
2021-06-04  6/4/2021     3206.22  2249749  3212.00  3221.00  3198.81
2021-06-03  6/3/2021     3187.01  2398293  3204.23  3214.44  3184.03


In [67]:
# Removing $ sign from data set
df['Close/Last'] = df['Close/Last'].apply(lambda x : str(x).strip('$') if '$' in str(x) else str(x))
df['Open'] = df['Open'].apply(lambda x : str(x).strip('$') if '$' in str(x) else str(x))
df['High'] = df['High'].apply(lambda x : str(x).strip('$') if '$' in str(x) else str(x))
df['Low'] = df['Low'].apply(lambda x : str(x).strip('$') if '$' in str(x) else str(x))

df['Close/Last'] = df['Close/Last'].apply(lambda x : float(x))
df['Open'] = df['Open'].apply(lambda x : float(x))
df['High'] = df['High'].apply(lambda x :float(x))
df['Low'] = df['Low'].apply(lambda x : float(x))

df = round(df,2)
# Remove the date and volume Column in data set
df=df.drop(columns=['Date'])
df=df.drop(columns=['Volume'])
# show the data
print(df.head())

# THERE WE BUILD OUR BUNDLE OF INDICATORS THAT WE ARE GOING TO USE AS INDEPENDENT VARIABLES
# 20 Days Moving average
df["S_20"]=df["Close/Last"].rolling(window=20).mean()

# What rolling() does :
# Calculate the moving average. That is, take the first 20 values, average
# them, then drop the 19 first and add the 20th newly calculated, etc.

# Rolling Correlation
# Here we take 20 days and calculate the correlation between the two features(Close and S_20) in the frame of these 20 days, then we
# iterate one row further for the next 20 days, etc...

df["Corr"]=df["Close/Last"].rolling(window=20).corr(df["S_20"])

# RSI measures speed of change of price movements. RSI value oscillates between 0 and 100. There is a rule
# of thumb that says that values above 70. indicate that the asset is being overbought.

# RSI
df["RSI"]=ta.RSI(np.array(df["Close/Last"]), timeperiod=20)

# The "shift" here indicates that we are subtracting the "open" of the actual day to the "close" of the previous day
df["Open-Close"]=df["Open"]-df["Close/Last"].shift(-1)
df["Open-Open"]=df["Open"]-df["Open"].shift(-1)

df=df.dropna()
print(df.head())

X=df.iloc[:,:9]

            Close/Last     Open     High      Low
Date                                             
2021-06-09     3281.15  3272.87  3297.58  3270.70
2021-06-08     3264.11  3222.61  3279.53  3218.01
2021-06-07     3198.01  3197.33  3208.00  3172.20
2021-06-04     3206.22  3212.00  3221.00  3198.81
2021-06-03     3187.01  3204.23  3214.44  3184.03
            Close/Last     Open     High      Low       S_20      Corr  \
Date                                                                     
2021-04-15     3379.09  3371.00  3397.00  3352.00  3342.7025  0.539792   
2021-04-14     3333.00  3404.04  3404.13  3326.00  3351.7555  0.424488   
2021-04-13     3400.00  3400.85  3432.00  3395.63  3360.5600  0.368299   
2021-04-12     3379.39  3355.21  3395.04  3351.15  3370.0050  0.216510   
2021-04-09     3372.20  3304.70  3372.20  3288.90  3374.0345  0.104509   

                  RSI  Open-Close  Open-Open  
Date                                          
2021-04-15  53.683739       38.00    

In [68]:
# "shift" allows us to subtract every open price with the close price of the previous day

# DEFINING THE DEPENDENT VARIABLE

# Here we take "-1" as parameter to the shift function as we want
# the closing previous day.
# The numpy (np) function "where" creates a new variable called "y" where
# the value is "1" if the close of the today is higher than the close of
# the previous day. In our case we create an array, but an additional column
# could have been created for example by replacing the "y" by "df['Target']"
# for example.

y=np.where(df["Close/Last"].shift(-1) < df["Close/Last"],1,-1)

# SPLITTING THE DATA SET
# We split the data set
# To do this we declare the variable "split" which is going to be an integer
split=int(0.8*len(df))
X_train, X_test, y_train, y_test=X[:split], X[split:], y[:split],  y[split:]

# Instantiate The Logistic Regression in Python

model = LogisticRegression(solver='lbfgs', max_iter=10000)
model = model.fit(X_train, y_train)

#LET’S INTERPRET THE COEFFICIENTS
coef=model.coef_

In [69]:

# The zip()  function creates the objects and that can be used to
# produce single item at a time. This function can create pandans DataFrames
# by merging two lists.

pd.DataFrame((zip(X.columns,np.transpose(model.coef_))),columns=['Variable','Coef'])

# CALCULATING CLASS PROBABILITIES
probability = model.predict_proba(X_test)
print(probability)

#Make predictions using the testing set
predicted = model.predict(X_test)
print(predicted)

# MODEL EVALUATION
# CONFUSION MATRIX
metrics.confusion_matrix(y_test, predicted)
# Classification Report
print(metrics.classification_report(y_test, predicted))

 # MODEL ACCURACY
model.score(X_test,y_test)
print(model.score(X_test,y_test))

[[9.99734150e-01 2.65850137e-04]
 [2.01287854e-04 9.99798712e-01]
 [8.46586649e-03 9.91534134e-01]
 [9.98028221e-01 1.97177868e-03]
 [2.33421749e-09 9.99999998e-01]
 [5.48583751e-01 4.51416249e-01]
 [9.83166404e-01 1.68335961e-02]
 [9.99999999e-01 6.69316926e-10]
 [1.09793477e-02 9.89020652e-01]
 [9.99715723e-01 2.84277442e-04]
 [9.15050258e-10 9.99999999e-01]
 [9.99830452e-01 1.69547677e-04]
 [9.98173943e-01 1.82605735e-03]
 [1.09381914e-06 9.99998906e-01]
 [3.89014368e-02 9.61098563e-01]
 [3.54402901e-06 9.99996456e-01]
 [4.79621073e-03 9.95203789e-01]
 [3.04375669e-02 9.69562433e-01]
 [6.86797279e-04 9.99313203e-01]
 [9.99761383e-01 2.38617408e-04]
 [2.84195867e-09 9.99999997e-01]
 [1.77379217e-07 9.99999823e-01]
 [9.99999998e-01 1.86524845e-09]
 [4.45070781e-07 9.99999555e-01]
 [9.99999984e-01 1.59713457e-08]
 [1.00000000e+00 9.23515685e-32]
 [5.12748288e-10 9.99999999e-01]
 [5.77787383e-01 4.22212617e-01]
 [4.34723990e-09 9.99999996e-01]
 [8.52220338e-06 9.99991478e-01]
 [2.910099

In [72]:
# Cross-Validation
cross_val=cross_val_score(LogisticRegression(solver='lbfgs', max_iter=10000), X, y, scoring="accuracy", cv=10)
print(cross_val)
print(cross_val.mean())

# Here I'll just create a data frame for it to be more clear
a = metrics.confusion_matrix(y_test, predicted)
b = pd.Series(['Actual : -1', 'Actual : 1'])
confusion_matrix = pd.DataFrame(a, columns=['Predicted : -1', 'Predicted : 1'])
confusion_matrix = confusion_matrix.set_index([b])
print(confusion_matrix)


[1.         0.99596774 0.99193548 0.99596774 1.         1.
 0.99596774 0.99595142 0.98785425 0.99190283]
0.9955547211701712
             Predicted : -1  Predicted : 1
Actual : -1             239              4
Actual : 1                0            253
